<a href="https://colab.research.google.com/github/albertopierini/CVProject_measure_fetal_head/blob/master/df_mask_rcnn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installazione dipendenze
 
#!git clone https://github.com/matterport/Mask_RCNN.git
#!pip install pycocotools
#!pip install mrcnn
 
#VERSIONI TENSORFLOW
#!pip install tensorflow
#!pip install tensorflow==1.4
#!pip install tensorflow-gpu==1.9.0
#!pip install tf-nightly-gpu
 
'''
!pip install tensorflow==1.15.2
!pip install tensorflow-gpu==1.15.2
!pip install keras==2.2.0
'''
# CONDA 10 & TF-GPU 1.14 (metodo 2)
#!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64 -O cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
#!dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
#!apt-key add /var/cuda-repo-10-0-local-10.0.130-410.48/7fa2af80.pub
#!apt-get update
#!apt-get install cuda
 
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14
 
 
!pip install keras==2.2.0
print("Installed packages")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
#Import dipendenze
import numpy
import scipy
import PIL
import tensorflow
import skimage
import h5py
import IPython
import cv2
import keras
import scipy
import matplotlib
import cython
import statistics

print("All dependencies are installed")

#Test ricerca GPU
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#import os
#os.chdir('/content/Mask_RCNN/samples')

In [ ]:
pip install pycocotools

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/mrcnn_training')

In [ ]:
pwd

In [ ]:
import sys
import random
import math
import numpy as np
import pandas as pd
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import os
from pycocotools.coco import COCO
from config import Config
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
DRIVE_DIR = '/content/gdrive/My Drive/CV Project/'

import warnings
warnings.filterwarnings("ignore")

 #Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
import utils
import model as modellib
import model2 as modellib2
import visualize
#Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
#import coco

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(DRIVE_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = "/content/gdrive/My Drive/CV Project/coco.h5"

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class FetalHeadConfiguration(Config):  
  NAME = "fetalHead"
  BACKBONE = "resnet101"
  GPU_COUNT = 1
  IMAGES_PER_GPU = 4
  NUM_CLASSES = 1 + 1 #Background + fetal head
  STEPS_PER_EPOCH = 369
  IMAGE_MIN_DIM = 512
  IMAGE_MAX_DIM = 512
  IMAGE_MIN_SCALE = 2.0
  MAX_GT_INSTANCES = 1
  #LEARNING_RATE = 0.001
  LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }
  DETECTION_MIN_CONFIDENCE = 0

config = FetalHeadConfiguration()
config.display()

In [ ]:
class FetalHeadDataset(utils.Dataset):

  def load_fetal_heads(self, read_path, df_path):
    self.add_class("fetal_head", 1, "fetal_head")
    #Read and iterate dataframe
    df = pd.read_csv(df_path, sep = ',')
    i = 0

    for index, row in df.iterrows():
      image_path = read_path + '/images/' + row['filename']
      mask_path = read_path + '/gauss_smussato/' + row['annotation']
      #image = skimage.io.imread(image_path)
      #height, width = image.shape[:2]
      self.add_image("fetal_head", image_id = i, filename = row['filename'], path = image_path, mask_path = mask_path)
      i += 1


  def load_mask(self, image_id):
    #carichiamo la mask e la trasformiamo
    image_info = self.image_info[image_id]
    mask_path = image_info['mask_path']

    mask_image = cv2.imread(mask_path,0)
    mask_image = mask_image/255


    #ret, binary_mask = cv2.threshold(mask_image_gray,127,1,cv2.THRESH_BINARY)
    mask_image = np.expand_dims(mask_image, axis=-1)
    return mask_image, np.ones([mask_image.shape[-1]], dtype=np.int32)

  def image_reference():
    """Return the path of the image."""
    info = self.image_info[image_id]
    if info["source"] == "fetal_head":
      return info["path"]
    else:
      super(self.__class__, self).image_reference(image_id)

In [ ]:
import model2 as modellib2

model2 = modellib2.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

model2.load_weights('/content/gdrive/My Drive/fetalhead20200620T1301/mask_rcnn_fetalhead_0022.h5', by_name=True)
#model2.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
from imgaug import augmenters as iaa

def train(model, read_path):
    """Train the model."""
    # Training dataset.
    dataset_train = FetalHeadDataset()
    df_train_path = '/content/gdrive/My Drive/CV Project/csv/traingauss.csv'
    dataset_train.load_fetal_heads(read_path, df_train_path)
    dataset_train.prepare()

    dataset_validation = FetalHeadDataset()
    df_validation_path = '/content/gdrive/My Drive/CV Project/csv/valgauss.csv'
    dataset_validation.load_fetal_heads(read_path, df_validation_path)
    dataset_validation.prepare()
    print('Dataset Validation pronto')

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    
    augmentation = iaa.Sequential([
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-10, 10),
                shear=(-2, 2))
            ], random_order=True)
    
    
    '''
    augmentation = iaa.Sequential([
            iaa.Fliplr(0.5), # horizontal flips
            iaa.Crop(percent=(0, 0.1)), # random crops
            # Small gaussian blur with random sigma between 0 and 0.5.
            # But we only blur about 50% of all images.
            iaa.Sometimes(0.5,
                iaa.GaussianBlur(sigma=(0, 0.5))
            ),
            # Strengthen or weaken the contrast in each image.
            iaa.ContrastNormalization((0.75, 1.5)),
            # Make some images brighter and some darker.
            # In 20% of all cases, we sample the multiplier once per channel,
            # which can end up changing the color of the images.
            iaa.Multiply((0.8, 1.2), per_channel=0),
            # Apply affine transformations to each image.
            # Scale/zoom them, translate/move them, rotate them and shear them.
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-10, 10),
                shear=(-2, 2))
            ], random_order=True)
    '''


    print("Train network heads")
    #model.train(dataset_train, dataset_validation,
                #learning_rate=config.LEARNING_RATE,
                #epochs=30,
                #augmentation=augmentation,
                #layers='heads')
    #print("Train network layers 4+")
    #model.train(dataset_train, dataset_validation,
     #           learning_rate=config.LEARNING_RATE,
      #          epochs=20,
       #         augmentation=augmentation,
        #        layers='4+')

    print("Train all layers")
    model.train(dataset_train, dataset_validation,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                augmentation=augmentation,
                layers='all')
    #model.keras_model.save_weights('/content/drive/My Drive/Progetto CV fetal head/pesi/mask_rcnn_fetal_head.h5')


In [ ]:
read_path = '/content/gdrive/My Drive/train_set'

train(model2, read_path)

In [ ]:
model2.keras_model.save_weights('/content/gdrive/My Drive/CV Project/pesi/mrcnn_pesi.h5')